In [3]:
# ЗАДАЧА 1: Image Classification
import time
from io import BytesIO

import requests
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Устройство
device = torch.device("cpu")

# Трансформации (одинаковые для всех моделей ImageNet)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Модели
models_dict = {
    "resnet50": models.resnet50(weights="IMAGENET1K_V2"),
    "mobilenet_v2": models.mobilenet_v2(weights="IMAGENET1K_V2"),
    "efficientnet_b0": models.efficientnet_b0(weights="IMAGENET1K_V1"),
}

# Переводим в eval и на устройство
for name, model in models_dict.items():
    model = model.to(device)
    model.eval()
    models_dict[name] = model

# Несколько картинок (публичные ссылки, рабочие на 2026)
image_urls = [
    "https://images.unsplash.com/photo-1583511655857-d19b40a7a54e?w=800&auto=format&fit=crop&q=80",
    "https://images.unsplash.com/photo-1615751072497-5f5169febe17?w=800&auto=format&fit=crop&q=80",
    "https://images.unsplash.com/photo-1560807707-8cc77767d783?w=800&auto=format&fit=crop&q=80",
    "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=800&auto=format&fit=crop&q=80",
    "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=800&auto=format&fit=crop&q=80",
]


# Классы ImageNet (можно загрузить, но для примера берём топ-1)
def get_top_pred(output, topk=1):
    probs = torch.nn.functional.softmax(output, dim=1)
    values, indices = probs.topk(topk)
    return indices[0].item(), values[0].item()


print("Модель\t\tКласс\tВероятность\tВремя (мс)")
print("-" * 60)

for url in image_urls[:5]:
    print(f"\nИзображение: {url.split('/')[-1][:10]}...")

    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    for name, model in models_dict.items():
        t0 = time.time()
        with torch.no_grad():
            out = model(input_tensor)
        dt = (time.time() - t0) * 1000

        cls, prob = get_top_pred(out)
        print(f"{name:<15}\t{cls:>4}\t{prob:.4f}\t{dt:.1f} ms")


# TODO:
# 1. Загрузить 2-3 разные pretrained модели (ResNet, MobileNet, EfficientNet)
# 2. Сделать inference на 5-10 картинках
# 3. Сравнить результаты и скорость
# 4. Понять: как выбирать модель для задачи

Модель		Класс	Вероятность	Время (мс)
------------------------------------------------------------

Изображение: photo-1583...
resnet50       	 245	0.4607	59.5 ms
mobilenet_v2   	 245	0.6492	16.3 ms
efficientnet_b0	 245	0.9903	22.4 ms

Изображение: photo-1615...
resnet50       	 157	0.1413	64.3 ms
mobilenet_v2   	 263	0.1586	19.4 ms
efficientnet_b0	 263	0.1698	27.5 ms

Изображение: photo-1560...
resnet50       	 156	0.4624	54.0 ms
mobilenet_v2   	 156	0.5564	14.9 ms
efficientnet_b0	 156	0.8555	19.4 ms

Изображение: photo-1506...
resnet50       	 970	0.1946	55.0 ms
mobilenet_v2   	 970	0.1569	15.0 ms
efficientnet_b0	 970	0.5285	18.7 ms

Изображение: photo-1514...
resnet50       	 285	0.1644	55.2 ms
mobilenet_v2   	 281	0.0631	12.5 ms
efficientnet_b0	 285	0.1959	19.6 ms


In [4]:
import torch
import torch.nn as nn

# Параметры
VOCAB_SIZE = 50000
EMB_DIM = 300
HIDDEN_DIM = 256
NUM_CLASSES = 2  # например positive / negative


# 1. Простая модель LSTM-классификатор
class TextLSTMClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(VOCAB_SIZE, EMB_DIM, padding_idx=0)
        self.lstm = nn.LSTM(EMB_DIM, HIDDEN_DIM, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(HIDDEN_DIM * 2, NUM_CLASSES)  # ×2 из-за bidirectional

    def forward(self, x):
        embeds = self.embedding(x)  # [batch, seq_len] → [batch, seq_len, emb]
        _, (hn, _) = self.lstm(embeds)  # берём последний hidden state
        hn = torch.cat((hn[-2, :, :], hn[-1, :, :]), dim=1)  # bidirectional
        out = self.fc(hn)
        return out


# Пример создания
model = TextLSTMClassifier()
print(model)

# 2. Загрузка GloVe (пример — torchtext уже не рекомендуется, лучше вручную)
# Скачать glove.6B.300d.txt → https://nlp.stanford.edu/projects/glove/
# Затем загрузить веса в embedding слой (очень упрощённо):

# embedding_matrix = torch.zeros(VOCAB_SIZE, EMB_DIM)
# ... (заполняем по словарю)
# model.embedding.weight.data.copy_(embedding_matrix)
# model.embedding.weight.requires_grad = False   # можно заморозить

TextLSTMClassifier(
  (embedding): Embedding(50000, 300, padding_idx=0)
  (lstm): LSTM(300, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)


In [5]:
import torchvision.models as models
import torch.nn as nn

# Берём предобученную модель
model = models.resnet18(weights="IMAGENET1K_V1")

# Замораживаем все параметры
for param in model.parameters():
    param.requires_grad = False

# Меняем последний слой под свою задачу (например 10 классов)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # ← только этот слой будет обучаться

# Альтернатива: размораживаем последние блоки
# for param in model.layer4.parameters():
#     param.requires_grad = True

# Теперь обучаем только fc (или fc + layer4)
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.001
)

print("Обучаемых параметров:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Обучаемых параметров: 5130
